In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import json
import logging
import pickle
import time
import datetime

In [2]:
conf = SparkConf().setAppName("KafkaExp")
conf = conf.setMaster('local[*]')
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/16 19:03:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
conf = SparkConf().setAppName("KafkaExp")
kafka_packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{"2.12"}:{"3.3.0"}',
    'org.apache.kafka:kafka-clients:2.8.0',
    "org.apache.hadoop:hadoop-aws:3.3.0",
    "com.google.guava:guava:21.0",
    "org.apache.httpcomponents:httpcore:4.4.8",
    "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1"
]
conf = conf.setMaster('local[*]').set('spark.jars.packages', ','.join(kafka_packages))
sc = SparkContext(conf=conf)

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.google.guava#guava added as a dependency
org.apache.httpcomponents#httpcore added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0aac1444-1ec0-4d73-9e8c-fae30fdf7ff0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#un

22/12/16 19:06:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
print(
    f'Loaded jars:\n{json.dumps((sc._jsc.sc().listJars().toList().toString().replace("List(", "").replace(")", "").split(", ")), indent=2)}')

Loaded jars:
[
  "spark://c901c2116227:32791/jars/commons-logging_commons-logging-1.1.3.jar",
  "spark://c901c2116227:32791/jars/com.google.guava_guava-21.0.jar",
  "spark://c901c2116227:32791/jars/com.amazonaws_aws-java-sdk-bundle-1.11.563.jar",
  "spark://c901c2116227:32791/jars/org.mongodb_mongodb-driver-core-4.0.5.jar",
  "spark://c901c2116227:32791/jars/org.apache.hadoop_hadoop-client-api-3.3.2.jar",
  "spark://c901c2116227:32791/jars/org.mongodb_bson-4.0.5.jar",
  "spark://c901c2116227:32791/jars/org.apache.httpcomponents_httpcore-4.4.8.jar",
  "spark://c901c2116227:32791/jars/org.mongodb.spark_mongo-spark-connector_2.12-3.0.1.jar",
  "spark://c901c2116227:32791/jars/org.apache.commons_commons-pool2-2.11.1.jar",
  "spark://c901c2116227:32791/jars/org.wildfly.openssl_wildfly-openssl-1.0.7.Final.jar",
  "spark://c901c2116227:32791/jars/org.apache.hadoop_hadoop-aws-3.3.0.jar",
  "spark://c901c2116227:32791/jars/org.spark-project.spark_unused-1.0.0.jar",
  "spark://c901c2116227:32791

In [5]:
spark = SparkSession(sc)

In [6]:
import datetime as dt
df = spark.read.option("header", True).csv("taxi_small.csv")

In [7]:
mongoURL = "mongodb://mongo:27017/transactions.data"
df.write.format("mongo").mode("append").option("uri",mongoURL).save()

In [5]:
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [8]:
df2 = spark.read.option("header", True).option("inferSchema","true").csv("taxi.csv")
df2.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [21]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,FloatType,DoubleType
schema = StructType().add("VendorID", StringType(), True)\
                    .add("total_amount",FloatType(),True)\
                    .add("trip_distancet",DoubleType(),True)

In [22]:
df3 = spark.read.option("header", True).schema(schema).csv("taxi.csv")
df3.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- trip_distancet: double (nullable = true)



In [13]:
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [7]:
mongoURL = "mongodb://mongo:27017/test.user"
df.write.format("mongo").mode("append").option("uri",mongoURL).save()

In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient("mongodb://mongo:27017/")

In [2]:
for db in client.list_databases():
    print(db)

{'name': 'admin', 'sizeOnDisk': 65536.0, 'empty': False}
{'name': 'config', 'sizeOnDisk': 8192.0, 'empty': False}
{'name': 'local', 'sizeOnDisk': 323584.0, 'empty': False}


In [ ]:
client.close()

In [17]:
df.groupBy("VendorID").count().show()

+--------+-----+
|VendorID|count|
+--------+-----+
|       1|   14|
|       2|    6|
+--------+-----+



MongoClient(host=['mongo:27017'], document_class=dict, tz_aware=False, connect=True)

In [10]:
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- RateCodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [17]:
conf = SparkConf().setAppName("test").setMaster("local[*]")

In [18]:
scc = SparkContext(conf=conf)

In [19]:
spark = SparkSession(scc)

In [9]:
schema = pickle.load(open("schema.pkl", 'rb'))

In [21]:
#query 1 .alias("language")

import datetime as dt
df = spark.read.schema(schema).option("multiline","true").json("data.json")
df.show()
# x = df.selectExpr("CAST(birth AS DATE)","CAST(death AS DATE)")

# y = x.withColumn('death', F.when(F.col('death').isNull(), dt.datetime.now().date()).otherwise(F.col('death')))
# y.show()
# z= y.withColumn("Age", F.year(F.col("death"))-F.year(F.col("birth")))
# z.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+------------+
|              awards|               birth|            contribs|               death|                name|       title|
+--------------------+--------------------+--------------------+--------------------+--------------------+------------+
|[{W.W. McDowell A...|1924-12-03T05:00:00Z|[Fortran, ALGOL, ...|2007-03-17T04:00:00Z|{null, John, Backus}|        null|
|[{Turing Award, A...|1927-09-04T04:00:00Z|[Lisp, Artificial...|2011-12-24T05:00:00Z|{null, John, McCa...|        null|
|[{Computer Scienc...|1906-12-09T05:00:00Z|[UNIVAC, compiler...|1992-01-01T05:00:00Z|{null, Grace, Hop...|Rear Admiral|
|[{Rosing Prize, N...|1926-08-27T04:00:00Z|       [OOP, Simula]|2002-08-10T04:00:00Z|{null, Kristen, N...|        null|
|[{Rosing Prize, N...|1931-10-12T04:00:00Z|       [OOP, Simula]|2002-06-29T04:00:00Z|{null, Ole-Johan,...|        null|
|[{Award for the A...|1956-01-31T05:00:0

In [11]:
# query 2

df.select(F.size(F.col("contribs")).alias("num_contribs")).show()

+------------+
|num_contribs|
+------------+
|           4|
|           3|
|           4|
|           2|
|           2|
|           1|
|           2|
|           1|
|           1|
|           1|
+------------+



In [12]:
# query 3
from pyspark.sql.types import ArrayType,StringType

def min_max(awards_year):
    mina = 10000
    maxa = 0
    # temp = []
    if awards_year is None:
        return []
    for year in awards_year:
        # temp.append(int(year))
        if int(year)>maxa:
            maxa = int(year)
            
        if int(year)<mina:
            mina = int(year)
    # ,[min(temp),max(temp)]
    return [mina,maxa]


convertUDF = F.udf(lambda z: min_max(z))
df.select(convertUDF(F.col("awards.year")).alias("min_max") ).show(truncate=False)

+------------+
|min_max     |
+------------+
|[1967, 1993]|
|[1971, 1990]|
|[1969, 1991]|
|[1999, 2001]|
|[1999, 2001]|
|[2001, 2003]|
|[1983, 2011]|
|[2011, 2011]|
|[2002, 2007]|
|[]          |
+------------+



In [13]:
df = df.selectExpr("name","contribs","awards","CAST(birth AS DATE)","CAST(death AS DATE)")
df = df.withColumn('death', F.when(F.col('death').isNull(), dt.datetime.now().date()).otherwise(F.col('death')))
df= df.withColumn("age", F.year(F.col("death"))-F.year(F.col("birth")))
final = df.select(F.col("name"), F.col("age"),F.size(F.col("contribs")).alias("num_contribs"), convertUDF(F.col("awards.year")).alias("min_max"))
final.show()


+--------------------+---+------------+------------+
|                name|age|num_contribs|     min_max|
+--------------------+---+------------+------------+
|{null, John, Backus}| 83|           4|[1967, 1993]|
|{null, John, McCa...| 84|           3|[1971, 1990]|
|{null, Grace, Hop...| 86|           4|[1969, 1991]|
|{null, Kristen, N...| 76|           2|[1999, 2001]|
|{null, Ole-Johan,...| 71|           2|[1999, 2001]|
|{null, Guido, van...| 66|           1|[2001, 2003]|
|{null, Dennis, Ri...| 70|           2|[1983, 2011]|
|{Matz, Yukihiro, ...| 57|           1|[2011, 2011]|
|{null, James, Gos...| 67|           1|[2002, 2007]|
|{null, Martin, Od...| 67|           1|          []|
+--------------------+---+------------+------------+



In [14]:
mongoURL = "mongodb://mongo:27017/test.user"

In [15]:
final.write.format("mongo").mode("append").option("uri",mongoURL).save()

In [12]:
final.write.mode("append").parquet("new2.parquet")

In [13]:
df = spark.read.parquet("new2.parquet")
df.show()

+--------------------+---+------------+------------+
|                name|age|num_contribs|     min_max|
+--------------------+---+------------+------------+
|{null, John, Backus}| 83|           4|[1967, 1993]|
|{null, John, McCa...| 84|           3|[1971, 1990]|
|{null, Grace, Hop...| 86|           4|[1969, 1991]|
|{null, Kristen, N...| 76|           2|[1999, 2001]|
|{null, Ole-Johan,...| 71|           2|[1999, 2001]|
|{null, Guido, van...| 66|           1|[2001, 2003]|
|{null, Dennis, Ri...| 70|           2|[1983, 2011]|
|{Matz, Yukihiro, ...| 57|           1|[2011, 2011]|
|{null, James, Gos...| 67|           1|[2002, 2007]|
|{null, Martin, Od...| 67|           1|          []|
+--------------------+---+------------+------------+

